In [1]:
import numpy

# -------------------------
# Feed-forward network
# -------------------------
class Network:

    def __init__(self,layers):
        self.layers = layers

    def forward(self,Z):
        for l in self.layers: Z = l.forward(Z)
        return Z

    def gradprop(self,DZ):
        for l in self.layers[::-1]: DZ = l.gradprop(DZ)
        return DZ

# -------------------------
# ReLU activation layer
# -------------------------
class ReLU:

    def forward(self,X):
        self.Z = X>0
        return X*self.Z

    def gradprop(self,DY):
        return DY*self.Z

# -------------------------
# Fully-connected layer
# -------------------------
class Linear:

    def __init__(self,name):
        self.W = numpy.loadtxt(name+'-W.txt')
        self.B = numpy.loadtxt(name+'-B.txt')

    def forward(self,X):
        self.X = X
        return numpy.dot(self.X,self.W)+self.B

    def gradprop(self,DY):
        self.DY = DY
        return numpy.dot(self.DY,self.W.T)

# -------------------------
# Sum-pooling layer
# -------------------------
class Pooling:

    def forward(self,X):
        self.X = X
        self.Y = 0.5*(X[:,::2,::2,:]+X[:,::2,1::2,:]+X[:,1::2,::2,:]+X[:,1::2,1::2,:])
        return self.Y

    def gradprop(self,DY):
        self.DY = DY
        DX = self.X*0
        for i,j in [(0,0),(0,1),(1,0),(1,1)]: DX[:,i::2,j::2,:] += DY*0.5
        return DX

# -------------------------
# Convolution layer
# -------------------------
class Convolution:

    def __init__(self,name):
        wshape = map(int,list(name.split("-")[-1].split("x")))
        self.W = numpy.loadtxt(name+'-W.txt').reshape(wshape)
        self.B = numpy.loadtxt(name+'-B.txt')

    def forward(self,X):

        self.X = X
        mb,wx,hx,nx = X.shape
        ww,hw,nx,ny = self.W.shape
        wy,hy       = wx-ww+1,hx-hw+1

        Y = numpy.zeros([mb,wy,hy,ny],dtype='float32')

        for i in range(ww):
            for j in range(hw):
                Y += numpy.dot(X[:,i:i+wy,j:j+hy,:],self.W[i,j,:,:])

        return Y+self.B

    def gradprop(self,DY):

        self.DY = DY
        mb,wy,hy,ny = DY.shape
        ww,hw,nx,ny = self.W.shape

        DX = self.X*0

        for i in range(ww):
            for j in range(hw):
                DX[:,i:i+wy,j:j+hy,:] += numpy.dot(DY,self.W[i,j,:,:].T)

        return DX


In [ ]:
import numpy,PIL,PIL.Image

lowest = -1.0
highest = 1.0

# --------------------------------------
# Sampling data
# --------------------------------------

def getMNISTsample(N=12,seed=None,path=''):

    fx = '%s/t10k-images-idx3-ubyte'%path
    ft = '%s/t10k-labels-idx1-ubyte'%path

    X  = numpy.fromfile(open(fx),dtype='ubyte',count=16+784*10000)[16:].reshape([10000,784])
    T  = numpy.fromfile(open(ft),dtype='ubyte',count=8+10000)[8:]
    T  = (T[:,numpy.newaxis]  == numpy.arange(10))*1.0

    if seed==None: seed=numpy.random
    else: seed=numpy.random.mtrand.RandomState(seed)

    R = seed.randint(0,len(X),[N])
    X,T = X[R],T[R]

    return X/255.0*(highest-lowest)+lowest,T

# --------------------------------------
# Color maps ([-1,1] -> [0,1]^3)
# --------------------------------------

def heatmap(x):

    x = x[...,numpy.newaxis]

    r = 0.9 - numpy.clip(x-0.3,0,0.7)/0.7*0.5
    g = 0.9 - numpy.clip(x-0.0,0,0.3)/0.3*0.5 - numpy.clip(x-0.3,0,0.7)/0.7*0.4
    b = 0.9 - numpy.clip(x-0.0,0,0.3)/0.3*0.5 - numpy.clip(x-0.3,0,0.7)/0.7*0.4

    return numpy.concatenate([r,g,b],axis=-1)

def graymap(x):

    x = x[...,numpy.newaxis]
    return numpy.concatenate([x,x,x],axis=-1)*0.5+0.5

# --------------------------------------
# Visualizing data
# --------------------------------------

def visualize(x,colormap,name):

    N = len(x); assert(N<=16)

    x = colormap(x/numpy.abs(x).max())

    # Create a mosaic and upsample
    x = x.reshape([1,N,28,28,3])
    x = numpy.pad(x,((0,0),(0,0),(2,2),(2,2),(0,0)),'constant',constant_values=1)
    x = x.transpose([0,2,1,3,4]).reshape([1*32,N*32,3])
    x = numpy.kron(x,numpy.ones([2,2,1]))

    PIL.Image.fromarray((x*255).astype('byte'),'RGB').save(name)



In [4]:
nn = Network([
    Linear('mlp/l1'),ReLU(),
    Linear('mlp/l2'),ReLU(),
    Linear('mlp/l3'),
])

In [7]:
class Network(Network):
    def relprop(self,R):
        for l in self.layers[::-1]: R = l.relprop(R)
        return R